<a href="https://colab.research.google.com/github/cccadet/dermatologist-ai/blob/master/Gradio_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install tensorflow-gpu
!pip install gradio
!pip install tfa-nightly

     |████████████████████████████████| 320.4MB 50kB/s 
     |████████████████████████████████| 983kB 4.1MB/s 
     |████████████████████████████████| 215kB 25.7MB/s 
     |████████████████████████████████| 962kB 27.1MB/s 
     |████████████████████████████████| 2.7MB 58.7MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 1.1MB 4.7MB/s 


In [1]:
import gradio as gr
import tensorflow as tf
import numpy as np
import requests
import os
import tensorflow_addons as tfa

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tfa.__version__

'0.12.0-dev'

In [2]:
download_path = './'
derm_ai_path = '/content/drive/My Drive/Derm AI/ISIC_2019'

In [3]:
%%time
print("Model2")
model2 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model2_InceptionResnetV2'))
print("Model5")
model5 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model5_Xception'))
print("Model6")
model6 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'B1-IRN50V2_3'))

Model2
Model5
Model6
CPU times: user 3min 45s, sys: 10.3 s, total: 3min 55s
Wall time: 4min 5s


In [4]:
model2._name="model2"
model5._name="model5"
model6._name="model6"

In [34]:
#personalização e união
x = tf.keras.layers.Input(shape=[299, 299, 3])

#model_1 = model1(x)
#model_1 = tf.math.multiply(model_1, [0.0067])

model_2 = model2(x)
model_2 = tf.math.multiply(model_2, [0.2630])

#model_3 = model3(x)
#model_3 = tf.math.multiply(model_3, [0.0019])

#model_4 = model4(x)
#model_4 = tf.math.multiply(model_4, [0.0205])

model_5 = model5(x)
model_5 = tf.math.multiply(model_5, [0.0446])

model_6 = model6(x)
model_6 = tf.math.multiply(model_6, [0.6925])

base_model1 = tf.math.add(model_2, model_5)
#base_model2 = tf.math.add(base_model1, model_3)
#base_model1 = tf.math.add(model_1, model_2)
#base_model2 = tf.math.add(base_model1, model_3)
#base_model3 = tf.math.add(base_model2, model_4)
#base_model4 = tf.math.add(base_model3, model_5)
prediction_layer =  tf.math.add(base_model1, model_6)

#prediction_layer = tf.keras.layers.Average()([model_1, model_2, model_4, model_5])

#prediction_layer = tf.keras.layers.Dense(units = 8, activation = "softmax")(base_model)

#final_model
model = tf.keras.models.Model(inputs = x, outputs = prediction_layer)

#optimizer
radam = tfa.optimizers.RectifiedAdam(
    lr=1e-2,
    total_steps=251,
    warmup_proportion=0.1,
    min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
optm = ranger

#compilador
model.compile(optimizer=optm, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

#model summary
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
model2 (Functional)             (None, 8)            54349032    input_3[0][0]                    
__________________________________________________________________________________________________
model5 (Functional)             (None, 8)            20877872    input_3[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Mul_3 (TensorFlowOp [(None, 8)]          0           model2[1][0]                     
_______________________________________________________________________________________



```
"AK":pred[:,0],
"BCC":pred[:,1],
"BKL":pred[:,2],
"DF":pred[:,3],
"MEL":pred[:,4],
"NV":pred[:,5],
"SCC":pred[:,6],
"VASC":pred[:,7],
```



In [36]:
labels = ['AK','BCC','BKL','DF','MEL','NV','SCC','VASC']
labels

['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']

In [37]:
def classify_image(inp):
  inp = inp.reshape((-1, 299, 299, 3))
  inp = tf.keras.applications.inception_resnet_v2.preprocess_input(inp)
  prediction = model.predict(inp)
  prediction = prediction[0]
  return {labels[i]: round(float(prediction[i]),5) for i in range(8)}

image = gr.inputs.Image(shape=(299, 299, 3))
label = gr.outputs.Label(num_top_classes=3)

gr.Interface(fn=classify_image, inputs=image, outputs=label).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on External URL: https://16700.gradio.app
Interface loading below...


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x7fb0e0378438>,
 'http://127.0.0.1:7878/',
 'https://16700.gradio.app')

In [ ]:
"""
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
import numpy as np

ORIGINAL = '/content/4.jpg'

DIM = 299

img = image.load_img(ORIGINAL, target_size=(DIM, DIM))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

def classify_image(inp):
  #inp = inp.reshape((-1, 299, 299, 3))
  prediction = model.predict(inp)
  prediction = prediction[0]
  return {labels[i]: round(float(prediction[i]),5) for i in range(8)}

classify_image(x)

"""
